In [ ]:
from disk_data_loader import load_from_file
import numpy as np

mnist=load_from_file('images.npydata', 'labels.npydata')
num_labels = 31*59
img_dim = [28,28]
img_vec_len = img_dim[0]*img_dim[1]

In [ ]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, img_vec_len])
y_ = tf.placeholder(tf.float32, shape=[None, num_labels])

W = tf.Variable(tf.zeros([img_vec_len,num_labels]))
b = tf.Variable(tf.zeros([num_labels]))

sess.run(tf.initialize_all_variables())

In [ ]:
y = tf.nn.softmax(tf.matmul(x,W) + b)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [ ]:
guess_label = np.zeros((100,num_labels))
true_label = np.zeros((100,num_labels))

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
train_accuracy_list = []
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
for i in range(100):
    batch = mnist.next_batch(1000)
    train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1]})
    train_accuracy_list.append(train_accuracy)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    
    guess_label = np.vstack((guess_label, y.eval(feed_dict={x: batch[0], y_: batch[1]})))
    true_label = np.vstack((true_label, batch[1]))

In [ ]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.images, y_: mnist.labels}))
guess_label_test = y.eval(feed_dict={x: mnist.images, y_: mnist.labels})

### Building Layers

In [ ]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [ ]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [ ]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,img_dim[0],img_dim[1],1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [ ]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [ ]:
W_fc1 = weight_variable([3136, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 3136])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [ ]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, num_labels])
b_fc2 = bias_variable([num_labels])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [ ]:
train_accuracy_list = []
guess_label = np.zeros((100,num_labels))
true_label = np.zeros((100,num_labels))
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log((y_conv)), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())

for i in range(1000):
    
    if i%100 == 10
        batch = mnist.next_batch(10000)
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        train_accuracy_list.append(train_accuracy)
        print("step %d, training accuracy %g"%(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
        guess_label = np.vstack((guess_label, y.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})))
        true_label = np.vstack((true_label, batch[1]))
        
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.images, y_: mnist.labels, keep_prob: 1.0}))

In [ ]:
testing_guess_label = y.eval(feed_dict={x: mnist.images, y_: mnist.labels})
np.save('training_guess_cnn.npy', testing_guess_label)
np.save('testing_guess_cnn.npy', guess_label)
np.save('testing_true_cnn.npy', true_label)
np.save('training_accuracy.npy', train_accuracy_list)